In [1]:
!pip install datasets
!pip install openpyxl
!pip install -q -U google-genai
# !pip install transformers
# !pip install accelerate
# !pip install peft
# !pip install bitsandbytes
# Imports
%load_ext autoreload
%autoreload 2

import pandas as pd
import sys

# Call models
from src.call_models import bedrock_connect, call_claude_bedrock
from src.call_models import google_connect, call_gemini, all_string_gemini_config, all_int_gemini_config
from src.translate_func import claude_translation, gemini_translation

# Datasets
from prompts import mmlu_prompts
from src.benchmarks_code import mmlu
# Access keys
from my_access_keys import google_access_key, aws_access_key, aws_secret_key

# .csv utils
from src.save_utils import add_dataset_to_csv

# Remove annoying warning
from IPython.core.display_functions import display

In [2]:
bedrock_client = bedrock_connect(aws_access_key, aws_secret_key)
google_client = google_connect(google_access_key)

In [6]:
mmlu_dataset = mmlu.get_mmlu_datasets()
mmlu_dataset['mmlu_dev'] = mmlu_dataset['mmlu_dev'].take(50)
mmlu_file_name = 'compare_csv/mmlu_train_top_50.csv'

print(mmlu_file_name)
mmlu_dataset

compare_csv/mmlu_train_top_50.csv


{'mmlu_dev': Dataset({
     features: ['question', 'subject', 'choices', 'answer'],
     num_rows: 50
 })}

In [7]:
mmlu_dataset

{'mmlu_dev': Dataset({
     features: ['question', 'subject', 'choices', 'answer'],
     num_rows: 50
 })}

In [8]:
df = add_dataset_to_csv(mmlu_file_name, 'original', mmlu_dataset['mmlu_dev'], mmlu.mmlu_sample_to_dict)
text_df = add_dataset_to_csv(mmlu_file_name[:-4] + '-text.csv', 'original', mmlu_dataset['mmlu_dev'], mmlu.mmlu_sample_to_dict)
display(df.head(2))
display(text_df.head(2))

,original
0,<question>Find all c in Z_3 such that Z_3[x]/(...
1,<question>Statement 1 | If aH is an element of...


,original
0,<question>Find all c in Z_3 such that Z_3[x]/(...
1,<question>Statement 1 | If aH is an element of...


In [9]:
exp_name = 'claude_v1_refine'
hebrew_datasets, text_output = claude_translation(
    bedrock_client,
    mmlu_dataset,
    mmlu_prompts.MMLU_INSTRUCT_V1_CLAUDE_REFINE,
    mmlu_prompts.MMLU_FEW_SHOTS,
    mmlu_prompts.MMLU_FORMAT,
    mmlu.mmlu_sample_to_dict,
    mmlu.mmlu_dict_to_sample,
)

Translating mmlu_dev...


  0%|          | 0/50 [00:00<?, ?it/s]

In [10]:
df = add_dataset_to_csv(mmlu_file_name, exp_name, hebrew_datasets['mmlu_dev'], mmlu.mmlu_sample_to_dict)
df.head(2)

,original,claude_v1_refine
0,<question>Find all c in Z_3 such that Z_3[x]/(...,<question>מצא את כל הערכים של c ב-Z_3 כך ש-Z_3...
1,<question>Statement 1 | If aH is an element of...,"<question>טענה 1 | אם aH הוא איבר בחבורת מנה, ..."


In [11]:
exp_name = 'gemini_v1_few_options'

hebrew_datasets = gemini_translation(
    google_client,
    mmlu_dataset,
    mmlu_prompts.MMLU_INSTRUCT_V1_GEMINI_MULTI,
    mmlu_prompts.MMLU_FEW_SHOTS,
    mmlu.mmlu_sample_to_dict,
    mmlu.mmlu_dict_to_sample,
    if_pro=False,
    think_bud=20_000
)

Translating mmlu_dev...


  0%|          | 0/50 [00:00<?, ?it/s]

In [18]:
df = add_dataset_to_csv(mmlu_file_name, exp_name, hebrew_datasets[0]['mmlu_dev'], mmlu.mmlu_sample_to_dict)
df.head(2)

,original,claude_v1_refine,gemini_v1_few_options
0,<question>Find all c in Z_3 such that Z_3[x]/(...,<question>מצא את כל הערכים של c ב-Z_3 כך ש-Z_3...,<question>מצא את כל c ב-Z_3 כך ש-Z_3[x]/(x^2 +...
1,<question>Statement 1 | If aH is an element of...,"<question>טענה 1 | אם aH הוא איבר בחבורת מנה, ...","<question>טענה 1 | אם aH הוא איבר בחבורת מנה, ..."


In [17]:
hebrew_datasets[0]['mmlu_dev']

Dataset({
    features: ['question', 'subject', 'choices', 'answer'],
    num_rows: 50
})